# Section I: Basic Array Operations

* To enter REPL, type `julia` in your terminal (PATH variable in shell should be set correctly).
* To get help in REPL, type `?` to enter `API` mode.
* To install a package, type `]` to enter `Pkg` mode. Type `?<Enter>` if you want some help in `Pkg` mode.
* To run shell command in REPL, type `;` to enter shell mode.
* To input `≈`, type `\approx<TAB>`, see full list of unicode input of julia [here](https://docs.julialang.org/en/v1/manual/unicode-input/).
* `@test` means stop program and throw and exception whenever a clause returns `false`. "function" start with `@` character is called a [macro](https://docs.julialang.org/en/v1/manual/metaprogramming/).

In [ ]:
# Julia Matrix Operation is fast. To know how fast it is, you need a benchmark package. Type `]` and `add BenchmarkTools<Enter>` in your REPL.
using BenchmarkTools, Test

cnot = [1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0]  # a matrix
b = [1, 1im, true, 0.4e2] # a vector
c = [1, 1im, 0.4e2, true] # a vector

# run tests
@test cnot*b ≈ c
@test_throws DimensionMismatch cnot*randn(3)  # expected to raise specific error

# run a benchmark
res1 = @benchmark $cnot*$b  # `$` sign means evaluation first, used in `@benchmark` to avoid taking time to evaluate cnot input account.

# Can it be faster? Yes if the matrix is small and static!

For static array, we can avoid all allocations.

### **Used in**
* Quantum Circuit Simulation

### **Challenge!**
Read the document of package `StaticArrays`
https://github.com/JuliaArrays/StaticArrays.jl
and show it is really fast!

In [ ]:
# install `StaticArrays`
# sa = <some static matrix> 
# sb = <some static vector> 

using StaticArrays: SMatrix, SVector
scnot = SMatrix{4, 4}(cnot)
sb = SVector{4}(b)


@testset "static arrays" begin
    @test scnot*sb ≈ c
    using Statistics: median
    res2 = @benchmark $scnot*$sb
    @test median(res2).time < median(res1).time / 5  # at least 5 times faster
end

# Linear Algebra

See [docs](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/).

### **Used in**
* Quantum Monte Carlo
* Non-Interacting fermionic systems (include mean field)

### **Challenge!**
Try to figure out how to use get eigenvalues, singular values and qr and pass the test

In [ ]:
using LinearAlgebra: eigen, svd, qr, I, det, tr
# hint, here you probabily want to type `?` and `eigen<Enter>` to get help in an REPL.

# sv_cnot = <get the singular value of matrix cnot>
# ev_cnot = <get the eigenvalue of matrix cnot>
# q_cnot = <get the Q matrix from QR decomposition of cnot>
# det_cnot = <get the determinant of matrix cnot>
# tr_cnot = <get the trace of matrix cnot>
sv_cnot = svd(cnot).S
ev_cnot = eigen(cnot).values
q_cnot = qr(cnot).Q
det_cnot = det(cnot)
tr_cnot = tr(cnot)

@testset "linalg" begin
    @test sv_cnot ≈ ones(4)
    @test ev_cnot ≈ [-1, 1, 1, 1]
    @test q_cnot*q_cnot' ≈ I
    @test det_cnot == -1
    @test tr_cnot == 2
end

# Large sparse matrix eigensolver

[KrylovKit](https://github.com/Jutho/KrylovKit.jl) is a package for solving large sparse matrix.


### **Used in**
* Exact diagonalization
* Cluster pertubation theory (CPT)
* Numerical Renomalization Group

### **Challenge!**
Get the lowest singular value of specific sparse matrix

In [ ]:
using SparseArrays: SparseMatrixCSC, sparse, nnz
sp = kron(SparseMatrixCSC(cnot), sparse(I, 100, 100))

# install `KrylovKit`
# ev = <get lowest eigenvalue of sp>
using KrylovKit: eigsolve
vals = eigsolve(sp, 1, :SR)[1][1]

@testset "sparse" begin
    @test sp |> nnz == 100*4    # here `x |> f` is same as calling f(x).
    @test vals[1] ≈ minimum(eigen(sp |> Matrix).values)
end

# Tensor contraction

[TensorOperations](https://github.com/Jutho/TensorOperations.jl) is a high performance package for tensor contractions.

### **Used in**
* Tensor Networks (TRG, DMRG,...)


### **Challenge!**
Try to calculate the following contraction
$C_{lj} = A_{i,j,k}B_{i,k,l}$

In [ ]:
A = randn(6, 10, 5)
B = randn(6, 5, 7)

# install TensorOperations, and using it
# @tensor C[a, b] := <specify contraction>

using TensorOperations
@tensor C[l,j] := A[i,j,k] * B[i,k,l]

@test C |> size == (7, 10)

# Save and load data

[DelimitedFiles](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/index.html) is the `txt` format save and load standard module.

[JLD2 and FileIO](https://github.com/simonster/JLD2.jl) uses HDF5 format to save and load, suited for large data file.


### **Challenge!**
Read out the saved data.

In [ ]:
using DelimitedFiles

a = randn(Float64, 3,3)
writedlm("data/_test.dat", a)
# b = <read data from file `data/_test.dat`>
b = readdlm("data/_test.dat")

# FileIO
# install FileIO and JLD2
using FileIO, JLD2
jldopen("data/_example.jld2", "w") do f
    f["A"] = a
end

# b_jld2 = <read data from file `data/_example.jld2`>
b_jld2 = load("data/_example.jld2")["A"]
@testset "file reading" begin
    @test b ≈ a
    @test b_jld2 ≈ a
end

# Section II: Road to Master

# Defining functions

We have [multiple dispatch](https://docs.julialang.org/en/v1/manual/functions/) + [type tree](https://docs.julialang.org/en/v1/manual/types/) instead of classes in object oriented design.

In [ ]:
"""This is a utility of showing subtype tree."""
function subtypetree(t, level=1, indent=4)
   level == 1 && println(t)
   for s in subtypes(t)
     println(join(fill(" ", level * indent)) * string(s))
     subtypetree(s, level+1, indent)
   end
end

In [ ]:
subtypetree(Number)

In [ ]:
dump(Array)

### **Challendge!**
Fix following tests

In [ ]:
@testset "types" begin
    @test 1.0 isa Float64
    @test 1.0 isa Real
    @test typeof(1.0) == Float64
    
    # type relation
    @test Int64 <: Int
    @test Int64 === Int
    @test Int64 <: Integer
    @test Int64 <: Union{Int64, Complex}
    @test Array{ComplexF64, 3} <: Array{ComplexF64}
    @test Array{ComplexF64, 3} <: Array{<:Complex, 3}
    @test supertype(Integer) == Real
    @test Signed in subtypes(Integer)
    
    # type promotion
    @test eltype(promote(1.0, 2im)) == ComplexF64
    @test promote_type(Float32, Float64) == Float64
    @test promote_type(Int64, Real, Float64) == Real
    
    # element types
    @test eltype([1, 2, 3.0]) == Float64
    @test eltype(Int[1, 2, 3.0]) == Int64
    
end

In [ ]:
# define function f
f(x) = x^2
function f(x::Vector{T}) where T  # `where` means we want to know the type parameter T.
    println("we get type `$T`!")
    append!(copy(x), x)
end
function f(x::Vector{<:Complex})
    append!(copy(x), x |> conj)
end
# inplace vector version of f
function f!(x::Vector)
    append!(x, x)
end
f(x::String) = x^2
macro f(x)
   return :($(x^2))
end

#=
    define function f here
=#
@testset "functions" begin
    # function and broadcast
    @test f(3) == 9
    @test f.([1,2,3,4,5]) == [1,4,9,16,25]  # broadcast
    
    # macro
    @test (@f 3) == 9
    #=
    var = 3
    @test (@f var) == 9     # try to uncomment this line!
    =#
    
    @test f("bili") == "bilibili"
    @test f.(["bili", "dili"]) == ["bilibili", "dilidili"]

    x = [1, 2, 3]
    y = [1im, 2+3im, 3]
    @test f(x) == [1,2,3,1,2,3]  # repeat the array
    @test f(y) == [1im,2+3im,3,-1im,2-3im,3]  # repeat, but with conjugate
    y = copy(x)
    @test (f!(y); y) == [1,2,3,1,2,3]  # inplace version of f
end

### **Challenge!**
Metaprogramming can be used to generate functions! Try to generate functions after reading [this section](https://docs.julialang.org/en/v1/manual/metaprogramming/#Generated-functions-1).

Metaprogramming is a **black technology** in Julia, especially important in multiple dispatch framework. It will save you time if used properly, but can also cause trouble in code readability (where is this function defined?).

In [ ]:
FUNCS = Symbol.('a' .+ (0:4))

for FUNC in FUNCS
    @eval $(Symbol(FUNC, :_gen))(x::Int) = x^2
end

@testset "generated functions" begin
    @test a_gen(3) == 9
    @test b_gen(3) == 9
    @test c_gen(3) == 9
    @test d_gen(3) == 9
    @test e_gen(3) == 9
end

# Type Stability

[Type stability](https://docs.julialang.org/en/v1/manual/performance-tips/index.html#Avoid-changing-the-type-of-a-variable-1) is the hardest and most important part of writing high performance Julia programs.

### **Challenge!**
Fix the allocation below to increase the performance of calculating Fibonacci.

In [ ]:
# fix Fibonacci to improve performance
function fib(n)
    a = 0
    b = 1
    for i = 1:n-1
        c = a + b
        a = b
        b = c
    end
    b
end
display(@benchmark fib(30))
display(@code_warntype fib(30))

In [ ]:
@testset "fibonacci" begin
    @test fib(30) == 832040
    @test (@allocated fib(30))  == 0
end

# Final Challenge

Give data file `example.txt`, with each row a tuple of $i, j, w_{ij}$.
The problems is to find the ground state configuration of the classical Ising hamiltonian $H = \sum\limits_{i,j} w_{ij}\sigma_i \sigma_j$

This is the code chanlledge of 2016 UCAS summer school.

### Simmulated Annealling
Fix the code of simulated annealing

1. correctify the code
2. improve the performance

In [ ]:
# please open `simulated_annealing.ipynb` to continue